In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
from Event import *

# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

openai = OpenAI()

# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are an entity extractor. When one asks you to create an event for him, you extract entities as start datetime, end datetime, summary, recurrence.
You answer only with a json built on the structure of a Google Calendar event. No explanation. No other words.
"""


def getUserPrompt(user_prompt):
    prompt = "You are an assistant that creates Google Calendar Events. Given this prompt:\n"
    prompt += user_prompt
    prompt += "\n you are supposed to return nothing else but a json formatted as a Google Calendar Event with the data given to the prompt."
    prompt += "\n If some crucial data is missing ask for it."
    return prompt
    

def getMessages(user_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": getUserPrompt(user_prompt)}
    ]

# And now: call the OpenAI API. You will get very familiar with this!

def askOpenAI(user_prompt):
    response = openai.chat.completions.create(
        model = "gpt-o1",
        messages = getMessages(user_prompt)
    )
    return response.choices[0].message.content

# A function to display this nicely in the Jupyter output, using markdown

def getOpenAIResponse(user_prompt):
    response = askOpenAI(user_prompt)
    display(response)


API key found and looks good so far!


In [2]:
event = Event()
correct_response = event.create_event()
print(getUserPrompt(event.prompt['en']))
print(getUserPrompt(event.prompt['gr']))


You are an assistant that creates Google Calendar Events. Given this prompt:
24 to 11 parent-teacher meeting every 13 of each month
 you are supposed to return nothing else but a json formatted as a Google Calendar Event with the data given to the prompt.
 If some crucial data is missing ask for it.
You are an assistant that creates Google Calendar Events. Given this prompt:
11 παρά εικοσιτέσσερα κάθε μήνα στις 13 συνάντηση γονέων-καθηγητών
 you are supposed to return nothing else but a json formatted as a Google Calendar Event with the data given to the prompt.
 If some crucial data is missing ask for it.


In [3]:
getOpenAIResponse(event.prompt['en'])

'{\n  "start": {\n    "dateTime": "2023-11-13T00:00:00",\n    "timeZone": "UTC"\n  },\n  "end": {\n    "dateTime": "2023-11-13T01:00:00",\n    "timeZone": "UTC"\n  },\n  "summary": "Parent-Teacher Meeting",\n  "recurrence": [\n    "RRULE:FREQ=MONTHLY;BYMONTHDAY=13"\n  ]\n}'

In [4]:
display(correct_response['en']) # 4-o-mini

{'summary': 'parent-teacher meeting',
 'start': {'dateTime': "2025-02-13T10:36:00+02:00', 'timeZone':'Eastern European Time',"},
 'end': {'dateTime': "2025-12-31T12:36:00+02:00', 'timeZone':'Eastern European Time',"},
 'recurrence': ["RRULE:{'FREQ': 'MONTHLY', 'BYMONTHDAY': 13, 'UNTIL': '20251231T000000Z'}"]}

In [5]:
getOpenAIResponse(event.prompt['gr'])

'```json\n{\n  "start": {\n    "dateTime": "2023-11-11T11:36:00+00:00"\n  },\n  "end": {\n    "dateTime": "2023-11-11T12:36:00+00:00"\n  },\n  "summary": "Συνάντηση γονέων-καθηγητών",\n  "recurrence": [\n    "RRULE:FREQ=MONTHLY;BYMONTHDAY=13"\n  ]\n}\n```'

In [6]:
getOpenAIResponse(event.prompt['en']) # 4-o

'```json\n{\n  "start": {\n    "dateTime": "2023-11-24T00:00:00",\n    "timeZone": "UTC"\n  },\n  "end": {\n    "dateTime": "2023-11-24T01:00:00",\n    "timeZone": "UTC"\n  },\n  "summary": "Parent-Teacher Meeting",\n  "recurrence": [\n    "RRULE:FREQ=MONTHLY;BYMONTHDAY=13"\n  ]\n}\n```'

In [7]:
getOpenAIResponse(event.prompt['gr'])

'```json\n{\n  "start": {\n    "dateTime": "2023-11-11T11:36:00",\n    "timeZone": "UTC"\n  },\n  "end": {\n    "dateTime": "2023-11-11T12:36:00",\n    "timeZone": "UTC"\n  },\n  "summary": "Συνάντηση γονέων-καθηγητών",\n  "recurrence": "RRULE:FREQ=MONTHLY;BYMONTHDAY=11"\n}\n```'

In [8]:
getOpenAIResponse(event.prompt['en']) # o1

'```json\n{\n  "start": {\n    "dateTime": "2023-11-24T09:00:00",\n    "timeZone": "UTC"\n  },\n  "end": {\n    "dateTime": "2023-11-24T10:00:00",\n    "timeZone": "UTC"\n  },\n  "summary": "Parent-Teacher Meeting",\n  "recurrence": [\n    "RRULE:FREQ=MONTHLY;BYMONTHDAY=13"\n  ]\n}\n```'

In [9]:
getOpenAIResponse(event.prompt['gr'])

'```json\n{\n  "summary": "Συνάντηση γονέων-καθηγητών",\n  "start": {\n    "dateTime": "2023-11-11T11:36:00",\n    "timeZone": "Europe/Athens"\n  },\n  "end": {\n    "dateTime": "2023-11-11T12:36:00",\n    "timeZone": "Europe/Athens"\n  },\n  "recurrence": [\n    "RRULE:FREQ=MONTHLY;BYMONTHDAY=13"\n  ]\n}\n```'